In [23]:
import pandas as pd
import numpy as np
import scipy.stats as sp
from numpy.random import Generator, SFC64
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected = True)
rg = Generator(SFC64())

In [3]:
def BSM(S0, K, tau, r, sigma, opt_type='c', q=0):
    d1 = (np.log(S0/K)+(r-q+.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    N = lambda x: sp.norm.cdf(x)
    if opt_type == 'c':
        return S0*np.exp(-q*tau)*N(d1) - np.exp(-r*tau)*K*N(d2)
    else:
        return K*np.exp(-r*tau)*N(-d2) - S0*np.exp(-q*tau)*N(-d1)

# Simulating Stock Process with Trees

In [88]:
r = 0.0013 # 1yr Treasury Rate for rf
def binomial_tree(S, K, T, r, sig, N, opt_type='c', am=False):
    # S is underlying price at time 0
    # K is strike price
    # T is time t maturity
    # r is risk free rate
    # sig is volatility of the underlying
    # N is the number of steps
    # opt_type is 'c' for call else put
    # am is True for American else European
    
    if opt_type == 'c': #c is multiplied by payout s - k so that it is negated to k - s for puts
        c = 1
    else:
        c = -1
    dt = T/N # time between each step used to discount from one step to the next
    v = r-.5*sig**2
    u = np.sqrt((v*dt)**2 + dt*sig**2)
    d = -u
    disc = np.exp(-r*dt)
    
    #initialize stock paths
    stock_price = np.ones((N+1,N+1))
    
    #set stock price to S at time 0
    stock_price[0,0] = S
    
    #stock price generation, first initialize down paths
    for i in range(1,N+1):
        stock_price[i,0] = stock_price[i-1,0]*np.exp(d)
        # for each down path assign up paths
        for j in range(1,i+1):
            stock_price[i,j] = stock_price[i-1,j-1]*np.exp(u)
    return stock_price

pd.DataFrame(binomial_tree(100,105,1,r,.20,10))

,0,1,2,3,4,5,6,7,8,9,10
0,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,93.868700,106.531784,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,88.113328,100.000000,113.490209,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,82.710835,93.868700,106.531784,120.903144,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,77.639585,88.113328,100.000000,113.490209,128.800276,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5,72.879269,82.710835,93.868700,106.531784,120.903144,137.213231,1.000000,1.000000,1.000000,1.000000,1.000000
6,68.410822,77.639585,88.113328,100.000000,113.490209,128.800276,146.175703,1.000000,1.000000,1.000000,1.000000
7,64.216349,72.879269,82.710835,93.868700,106.531784,120.903144,137.213231,155.723583,1.000000,1.000000,1.000000
8,60.279052,68.410822,77.639585,88.113328,100.000000,113.490209,128.800276,146.175703,165.895111,1.000000,1.000000
9,56.583162,64.216349,72.879269,82.710835,93.868700,106.531784,120.903144,137.213231,155.723583,176.731021,1.000000


In [7]:
steps = 10
temp = pd.DataFrame(binomial_tree(100,105,1,r,.20,steps,'c',True))
data = []
for i in range(0,steps+1):
    trace = go.Scatter(
        x = (steps+1)*[i], y = temp.iloc[i][temp.iloc[i] != 1], name = 'Step {}'.format(i), mode = 'markers'
    )
    data.append(trace)
layout = go.Layout(
    title = 'Stock Price Binomial Tree', yaxis = dict(title = 'Stock Price ($)'), 
        xaxis = dict(title = 'Time Step'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [118]:
Ns = np.linspace(10,500,50)
trace0 = go.Scatter(
    x = Ns, y = [binomial_tree(100,105,1,r,.20,int(n)) for n in Ns], name = 'Binomial Tree'
)
trace1 = go.Scatter(
    x = Ns, y = len(Ns)*[BSM(100, 105, 1, r, .20)], name = 'Black-Scholes'
)
data = [trace0,trace1]
layout = go.Layout(
    title = 'Stock Price Binomial Tree', yaxis = dict(title = 'Stock Price ($)'), 
        xaxis = dict(title = 'Time Step'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [14]:
temp.iloc[10].mean()

108.19933389749426

# Building Trees to Model Option Value

In [17]:
def binomial_tree(S, K, T, r, sig, N, opt_type='c', am=False):
    # S is underlying price at time 0
    # K is strike price
    # T is time t maturity
    # r is risk free rate
    # sig is volatility of the underlying
    # N is the number of steps
    # opt_type is 'c' for call else put
    # am is True for American else European
    
    if opt_type == 'c': #c is multiplied by payout s - k so that it is negated to k - s for puts
        c = 1
    else:
        c = -1
    dt = T/N # time between each step used to discount from one step to the next
    v = r-.5*sig**2
    u = np.sqrt((v*dt)**2 + dt*sig**2)
    d = -u
    pu = .5+.5*(v*dt)/u
    pd = 1-pu
    disc = np.exp(-r*dt)
    
    #initialize stock and option paths
    stock_price = np.ones((N+1,N+1))
    opt_val = np.ones((N+1,N+1))
    
    #set stock price to S at time 0
    stock_price[0,0] = S
    
    #stock price generation, first initialize down paths
    for i in range(1,N+1):
        stock_price[i,0] = stock_price[i-1,0]*np.exp(d)
        # for each down path assign up paths
        for j in range(1,i+1):
            stock_price[i,j] = stock_price[i-1,j-1]*np.exp(u)
    
    #terminal option values derived from final stock prices in each path
    opt_val[N,:] = np.maximum(0,c*(stock_price[N,:]-K))
    
    #Recursion For Option Price
    #similar to stock paths but working backwards to calculate option values until time 0 is reached
    for i in range(N-1,-1,-1):
        for j in range(i+1):
            opt_val[i,j] = disc*(pu*opt_val[i+1,j+1]+
                                 pd*opt_val[i+1,j])
            if am: #adding exercise optionality for american options
                opt_val[i,j] = max(opt_val[i,j],
                                   c*(stock_price[i,j]-K))
    return opt_val
pd.DataFrame(binomial_tree(100,105,1,r,.20,10))

,0,1,2,3,4,5,6,7,8,9,10
0,6.129688,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,3.380566,9.047898,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.567760,5.304693,13.021521,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,0.549505,2.648457,8.124139,18.220687,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,0.110720,1.015162,4.381944,12.096443,24.722834,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5,0.000000,0.228214,1.850314,7.068946,17.433400,32.462882,1.000000,1.000000,1.000000,1.000000,1.000000
6,0.000000,0.000000,0.470389,3.314785,11.053664,24.206390,41.230967,1.000000,1.000000,1.000000,1.000000
7,0.000000,0.000000,0.000000,0.969555,5.803747,16.626309,32.254651,50.765067,1.000000,1.000000,1.000000
8,0.000000,0.000000,0.000000,0.000000,1.998425,9.842402,23.827871,41.203338,60.922792,1.000000,1.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,4.119108,15.916934,32.227040,50.737413,71.744875,1.000000


In [18]:
steps = 10
temp = pd.DataFrame(binomial_tree(100,105,1,r,.20,steps,'c',True))
data = []
for i in range(0,steps+1):
    trace = go.Scatter(
        x = (steps+1)*[i], y = temp.iloc[i][temp.iloc[i] != 1], name = 'Step {}'.format(i), mode = 'markers'
    )
    data.append(trace)
layout = go.Layout(
    title = 'Option Value Binomial Tree', yaxis = dict(title = 'Option Value ($)'), 
        xaxis = dict(title = 'Time Step'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [107]:
def binomial_tree(S, K, T, r, sig, N, opt_type='c', am=False):
    # S is underlying price at time 0
    # K is strike price
    # T is time t maturity
    # r is risk free rate
    # sig is volatility of the underlying
    # N is the number of steps
    # opt_type is 'c' for call else put
    # am is True for American else European
    
    if opt_type == 'c': #c is multiplied by payout s - k so that it is negated to k - s for puts
        c = 1
    else:
        c = -1
    dt = T/N # time between each step used to discount from one step to the next
    v = r-.5*sig**2
    u = np.sqrt((v*dt)**2 + dt*sig**2)
    d = -u
    pu = .5+.5*(v*dt)/u
    pd = 1-pu
    disc = np.exp(-r*dt)
    
    #initialize stock and option paths
    stock_price = np.ones((N+1,N+1))
    opt_val = np.ones((N+1,N+1))
    
    #set stock price to S at time 0
    stock_price[0,0] = S
    
    #stock price generation, first initialize down paths
    for i in range(1,N+1):
        stock_price[i,0] = stock_price[i-1,0]*np.exp(d)
        # for each down path assign up paths
        for j in range(1,i+1):
            stock_price[i,j] = stock_price[i-1,j-1]*np.exp(u)
    
    #terminal option values derived from final stock prices in each path
    opt_val[N,:] = np.maximum(0,c*(stock_price[N,:]-K))
    
    #Recursion For Option Price
    #similar to stock paths but working backwards to calculate option values until time 0 is reached
    for i in range(N-1,-1,-1):
        for j in range(i+1):
            opt_val[i,j] = disc*(pu*opt_val[i+1,j+1]+
                                 pd*opt_val[i+1,j])
            if am: #adding exercise optionality for american options
                opt_val[i,j] = max(opt_val[i,j],
                                   c*(stock_price[i,j]-K))
    return opt_val[0,0]
binomial_tree(100,105,1,r,.20,10)

6.1296876982036945

In [21]:
sig = 0.2
S = 100
K = 105
T = 1
def MC(S,K,r,sig,T,n,m,q=0,opt_type='c',print_df=False):
    # start time
    if print_df:
        start = time.time()
    # define dt and discount rate
    dt = T/n
    disc = np.exp(-r*T)
    c=1 if opt_type=='c' else -1
    # generate m arrays of random intervals of BM of length n with mean 0 variance dt
    dWs = np.array(np.split(rg.normal(0,np.sqrt(dt),n*m),m))
    # create arrays of stock paths begnning arrays of the differentials
    S_arr = np.cumsum((r-q - .5*sig**2)*dt+sig*dWs,axis=1)
    # add the log of the stock price then input into exponential function for array of stock values
    S_arr += np.log(S)
    S_arr = np.exp(S_arr)
    # array of m stock values at maturity T
    S_T = S_arr[:,n-1]
    opt_values = disc*np.maximum(c*(S_T - K),0)
    if print_df:
        run_time = time.time() - start
        return pd.DataFrame({'Option Type': opt_type, 'Sims (m)': m, 'Steps (n)': n,
                             'Option Value': opt_values.mean(), 'SE': opt_values.std()/np.sqrt(m), 
                             'Time': run_time}, index = [1])
    else:
        return opt_values.mean()

In [50]:
def MC_stock(S,K,r,sig,T,n,m,q=0,opt_type='c'):
    dt = T/n
    disc = np.exp(-r*T)
    c=1 if opt_type=='c' else -1
    # generate m arrays of random intervals of BM of length n with mean 0 variance dt
    dWs = np.array(np.split(rg.normal(0,np.sqrt(dt),n*m),m))
    # create arrays of stock paths begnning arrays of the differentials
    S_arr = np.cumsum((r-q - .5*sig**2)*dt+sig*dWs,axis=1)
    # add the log of the stock price then input into exponential function for array of stock values
    S_arr += np.log(S)
    S_arr = np.exp(S_arr)
    # array of m stock values at maturity T
    S_T = S_arr[:,n-1]
#     return pd.DataFrame(S_arr)
#     return S_T.mean(),S_T.std()
    return S_arr
MC_stock(100,105,r,.20,1,500,100)

array([[100.17723382,  99.49759188, 100.2636937 , ..., 115.01288585,
        115.68351553, 114.47634737],
       [ 99.43186553,  98.73414215,  98.84517283, ..., 110.61792845,
        110.31204932, 110.20201739],
       [ 99.88398341,  99.34333337, 100.17186204, ...,  78.83195178,
         78.73828495,  78.40339154],
       ...,
       [ 98.49068581, 100.60938805, 102.09373613, ..., 131.71163577,
        132.30340624, 132.78820134],
       [ 99.60454323, 101.71079875, 101.42366546, ..., 118.25016348,
        117.53074152, 117.89575198],
       [100.45135699, 101.48457232, 102.4662021 , ...,  78.88383291,
         78.91657212,  78.9585853 ]])

In [33]:
sims_df = pd.DataFrame(columns=['M','Mean','Standard Dev'])
for m in [1000,10000,100000,1000000]:
    temp_mean,temp_std = MC_stock(100,105,r,.20,1,500,m)
    sims_df = sims_df.append(pd.DataFrame({'M': m, 'Mean': temp_mean, 'Standard Dev': temp_std},index=[1]),
                             ignore_index=True)

sims_df.set_index('M')

,Mean,Standard Dev
M,,
1000,101.071402,20.968972
10000,99.751453,19.682062
100000,99.974975,20.174169
1000000,100.095249,20.229368


In [83]:
temp = MC_stock(100,105,r,.20,1,500,10000)
perc_df = pd.DataFrame(columns=['Percentile','Step 1'])
for p in range(101):
    perc_df = perc_df.append(pd.DataFrame({'Percentile':p,'Step 1':np.percentile(temp[:,0],p)},index=[1])
                             ,ignore_index=True)
perc_df = perc_df.set_index('Percentile')

In [84]:
for s in range(1,500):
    lst = []
    for p in range(101):
        lst.append(np.percentile(temp[:,s],p))
    perc_df['Step {}'.format(s+1)] = lst
    
perc_df

,Step 1,Step 2,Step 3,Step 4,Step 5,Step 6,Step 7,Step 8,Step 9,Step 10,...,Step 491,Step 492,Step 493,Step 494,Step 495,Step 496,Step 497,Step 498,Step 499,Step 500
Percentile,,,,,,,,,,,,,,,,,,,,,
0,96.094009,94.314591,94.377918,92.450187,92.586536,92.339100,92.035526,90.641120,90.799263,90.643074,...,47.207820,47.633591,48.219450,48.202260,48.305413,48.093045,48.050880,48.233965,48.131163,48.032900
1,97.914307,97.104323,96.452081,95.856335,95.476790,94.902426,94.604408,94.385022,94.153731,93.576594,...,61.914227,61.809956,61.628112,61.657112,61.604922,61.411701,61.456207,61.315092,61.372765,61.273860
2,98.196195,97.447707,96.835952,96.310861,95.907485,95.483295,95.217049,94.893539,94.654083,94.342467,...,65.071230,65.293357,65.287240,65.221342,65.178609,65.182785,65.140406,65.112174,65.085825,65.004058
3,98.323127,97.647121,97.087038,96.620331,96.270925,95.880742,95.618459,95.340487,95.053200,94.815916,...,67.604439,67.601721,67.565378,67.432240,67.398670,67.505595,67.383456,67.149899,67.235889,67.216418
4,98.429625,97.811684,97.281722,96.858509,96.502077,96.181732,95.904505,95.628409,95.415607,95.122905,...,69.117863,69.139504,69.064508,69.041864,69.115590,68.980569,68.939833,68.982306,68.930920,68.864509
5,98.524189,97.944211,97.418889,97.057119,96.679228,96.408022,96.151382,95.878927,95.650654,95.396716,...,70.614951,70.546421,70.628509,70.651980,70.693179,70.617536,70.659931,70.566566,70.480832,70.495116
6,98.603766,98.061215,97.562665,97.190635,96.866890,96.595910,96.349542,96.088342,95.888555,95.641458,...,71.843942,71.959479,71.900763,71.938197,72.005805,71.980342,71.958747,71.904410,71.824603,71.893795
7,98.678931,98.137350,97.687466,97.347173,97.033100,96.802471,96.546241,96.281805,96.091989,95.858063,...,73.054648,73.149185,73.144825,73.147338,73.154942,73.112376,73.111730,73.044160,73.167471,73.067814
8,98.749899,98.224240,97.802182,97.484058,97.171312,96.948164,96.711336,96.450529,96.262655,96.071377,...,74.146048,74.169467,74.146774,74.152911,74.231200,74.152923,74.123669,74.031720,74.106374,74.028960


In [92]:
perc_df.iloc[0:,499]
steps = 10
temp = pd.DataFrame(binomial_tree(100,105,1,r,.20,steps,'c',True))
data = []
for i in range(0,steps+1):
    trace = go.Scatter(
        x = 101*[i], y = perc_df.iloc[0:,np.minimum(i*50,499)], 
        name = 'MC Step {}'.format(i), mode='lines+markers'
    )
    data.append(trace)
    trace = go.Scatter(
        x = (steps+1)*[i], y = temp.iloc[i][temp.iloc[i] != 1], name = 'Step {}'.format(i), mode = 'markers'
    )
    data.append(trace)
layout = go.Layout(
    title = 'Stock Price Binomial Tree', yaxis = dict(title = 'Stock Price ($)'), 
        xaxis = dict(title = 'Time Step'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [106]:
MC(100,105,r,.20,1,500,100000)

5.937932759811109

In [109]:
binomial_tree(100,105,1,r,.20,500)

5.95793139023971

In [110]:
BSM(100, 105, 1, r, .20)

5.955610639997737

array([ 10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 110.,
       120., 130., 140., 150., 160., 170., 180., 190., 200., 210., 220.,
       230., 240., 250., 260., 270., 280., 290., 300., 310., 320., 330.,
       340., 350., 360., 370., 380., 390., 400., 410., 420., 430., 440.,
       450., 460., 470., 480., 490., 500.])

In [97]:
perc_df.iloc[0:,499]
steps = 500
temp = pd.DataFrame(binomial_tree(100,105,1,r,.20,steps,'c',True))
data = []
for i in range(11):
    trace = go.Scatter(
        x = (steps+1)*[i*50], y = temp.iloc[i*50][temp.iloc[i*50] != 1],name = 'Step {}'.format(i),mode = 'markers'
    )
    data.append(trace)
    trace = go.Scatter(
        x = 101*[i*50], y = perc_df.iloc[0:,np.minimum(i*50,499)], 
        name = 'MC Step {}'.format(i), mode='lines+markers'
    )
    data.append(trace)
layout = go.Layout(
    title = 'Stock Price Binomial Tree', yaxis = dict(title = 'Stock Price ($)'), 
        xaxis = dict(title = 'Time Step'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)